In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws

# Step 1: Initialize Spark Session with Hadoop-AWS connector
print("🚀 Initializing Spark session...")
spark = SparkSession.builder \
    .appName("ConnectToS3") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1") \
    .getOrCreate()

# Step 2: Configure Spark to use s3a and AWS credentials
print("🔐 Configuring Spark to use AWS credentials...")
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.aws.credentials.provider", "com.amazonaws.auth.profile.ProfileCredentialsProvider")
hadoop_conf.set("fs.s3a.endpoint", "s3.us-west-2.amazonaws.com") 

🚀 Initializing Spark session...
🔐 Configuring Spark to use AWS credentials...


In [65]:
channel_name = "esl_dota2"
chat_year = "2025"

In [66]:
parquet_file_path = f"s3a://twitch-emotes-analytics-project/data/gold/{channel_name}/{chat_year}/all_data_parquet"

In [67]:
df = spark.read.parquet(parquet_file_path)

In [68]:
cols_to_select = ["i_username", "t_timestamp", "i_badge_names", "i_badge_titles", "i_badge_versions"]

In [69]:
badge_info = df.select(cols_to_select)

In [70]:
from pyspark.sql.functions import explode, size

In [71]:
badge_info_with_count = badge_info.withColumn("badges_count", size("i_badge_names"))

In [72]:
badge_info_with_count.printSchema()

root
 |-- i_username: string (nullable = true)
 |-- t_timestamp: timestamp (nullable = true)
 |-- i_badge_names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- i_badge_titles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- i_badge_versions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- badges_count: integer (nullable = false)



In [78]:
badge_info_with_count.show(100)

+-------------------+--------------------+--------------------+--------------------+----------------+------------+
|         i_username|         t_timestamp|       i_badge_names|      i_badge_titles|i_badge_versions|badges_count|
+-------------------+--------------------+--------------------+--------------------+----------------+------------+
|            ch1ll1e|2025-03-02 11:01:...|           [moments]|[Moments Badge - ...|             [2]|           1|
|         yaboku1212|2025-03-02 11:02:...|           [moments]|[Moments Badge - ...|             [1]|           1|
|          the69joke|2025-03-02 11:02:...|           [moments]|[Moments Badge - ...|             [2]|           1|
|             moobot|2025-03-02 11:03:...|[moderator, partner]|[Moderator, Verif...|          [1, 1]|           2|
|             moobot|2025-03-02 11:03:...|[moderator, partner]|[Moderator, Verif...|          [1, 1]|           2|
|           luistadi|2025-03-02 11:03:...|                NULL|                N

In [73]:
badge_info_with_count.createOrReplaceTempView("badges")

In [93]:
explore_query = """
select i_username, i_badge_names, i_badge_titles, i_badge_versions
from badges
where i_username in (select distinct i_username
from badges
where badges_count = -1
limit 10)
order by badges_count desc
LIMIT 10
"""

In [95]:
example_1 = spark.sql(explore_query)

In [104]:
example_2 = spark.sql("""select i_username, i_badge_names, i_badge_titles, i_badge_versions
from badges
where i_username in (select distinct i_username
from badges
where i_badge_names is NULL
limit 10)
or i_username != 'b3arly'
order by badges_count desc
LIMIT 10""")

In [105]:
example_2.show()

+------------+--------------------+--------------------+----------------+
|  i_username|       i_badge_names|      i_badge_titles|i_badge_versions|
+------------+--------------------+--------------------+----------------+
|eunoobslayer|[predictions, sub...|[Predicted Pink (...|  [pink-2, 0, 1]|
|eunoobslayer|[predictions, sub...|[Predicted Pink (...|  [pink-2, 0, 1]|
|eunoobslayer|[predictions, sub...|[Predicted Pink (...|  [pink-2, 0, 1]|
|eunoobslayer|[predictions, sub...|[Predicted Pink (...|  [pink-2, 0, 1]|
|eunoobslayer|[predictions, sub...|[Predicted Pink (...|  [pink-2, 0, 1]|
|eunoobslayer|[predictions, sub...|[Predicted Pink (...|  [pink-2, 0, 1]|
|eunoobslayer|[predictions, sub...|[Predicted Pink (...|  [pink-2, 0, 1]|
|eunoobslayer|[predictions, sub...|[Predicted Pink (...|  [pink-2, 0, 1]|
|eunoobslayer|[predictions, sub...|[Predicted Pink (...|  [pink-2, 0, 1]|
|eunoobslayer|[predictions, sub...|[Predicted Pink (...|  [pink-2, 0, 1]|
+------------+--------------------+---

In [109]:
# explore_query = """
# select * from badges limit 10
# """

explore_query = """
with cte as (
select *, case when i_badge_names is NULL then 0 else badges_count end as badges_count_tf
from badges)
select *
from cte
"""

In [111]:
final_df = spark.sql(explore_query)

In [115]:
final_df.createOrReplaceTempView("badges")

In [116]:
query = """
SELECT i_username, max(badges_count_tf) as badge_count
FROM badges
GROUP BY i_username
ORDER BY badge_count DESC
"""

In [117]:
distinct_users_and_their_badge_count = spark.sql(query)

In [118]:
distinct_users_and_their_badge_count.count()

14998

In [119]:
distinct_users_and_their_badge_count.show()

+------------------+-----------+
|        i_username|badge_count|
+------------------+-----------+
|           demonek|          3|
|           merck2d|          3|
|          tacofps_|          3|
|jossilynwestingesq|          3|
|          ystamp19|          3|
|           apenman|          3|
|       astris_lumi|          3|
|            crtvly|          3|
|              oak0|          3|
|           xd3m1nx|          3|
|           rykhasa|          3|
|      faelullmarin|          3|
|            swamta|          3|
|        ganbayar13|          3|
|       ghostxking_|          3|
|      bruhungstein|          3|
|      eunoobslayer|          3|
|       zentumaisle|          2|
|          esownzxc|          2|
|        kazumaa007|          2|
+------------------+-----------+
only showing top 20 rows



In [120]:
distinct_users_and_their_badge_count.createOrReplaceTempView("pie")

In [129]:
pie_query = """
select badge_count as number_of_badges_per_user, count(i_username) as users_volume_count, 
        round(100*count(i_username)/(select count(*) from pie),2) as users_volume_percentage
from pie
group by badge_count
order by number_of_badges_per_user """

In [134]:
spark.sql(pie_query).write.option("header", True).csv("pie")

In [132]:
pwd

'/Users/hari14/Desktop/PHD/twitch_emotes_analytics_project/python_notebooks'

In [133]:
ls

explore_badgesinfo_jun25.ipynb  week4/
pie.csv/                        week5/
utilities/                      week6/


In [16]:
df_badge_info_flattened = badge_info.withColumn("badge_names", explode("i_badge_names")) \
                                    .withColumn("badge_titles", explode("i_badge_titles")) \
                                    .withColumn("badge_versions", explode("i_badge_versions"))

In [17]:
df_badge_info_flattened.show()

+--------------------+--------------------+----------------+-----------+--------------------+--------------+
|       i_badge_names|      i_badge_titles|i_badge_versions|badge_names|        badge_titles|badge_versions|
+--------------------+--------------------+----------------+-----------+--------------------+--------------+
|           [moments]|[Moments Badge - ...|             [2]|    moments|Moments Badge - T...|             2|
|           [moments]|[Moments Badge - ...|             [1]|    moments|Moments Badge - T...|             1|
|           [moments]|[Moments Badge - ...|             [2]|    moments|Moments Badge - T...|             2|
|[moderator, partner]|[Moderator, Verif...|          [1, 1]|  moderator|           Moderator|             1|
|[moderator, partner]|[Moderator, Verif...|          [1, 1]|  moderator|           Moderator|             1|
|[moderator, partner]|[Moderator, Verif...|          [1, 1]|  moderator|            Verified|             1|
|[moderator, partne

In [19]:
from pyspark.sql import functions as F

In [26]:
badge_names = df_badge_info_flattened.groupBy("badge_names").agg(F.count("badge_names").alias("tot_count")).sort("tot_count", ascending = 0)

In [36]:
badge_titles = df_badge_info_flattened.groupBy("badge_titles").agg(F.count("badge_titles").alias("tot_count")).sort("tot_count", ascending = 0)

In [37]:
print(badge_names.count())
print(badge_titles.count())

52


60


In [35]:
badge_names.show(100)

+--------------------+---------+
|         badge_names|tot_count|
+--------------------+---------+
|             moments|    65465|
|         predictions|    33909|
|           moderator|     8563|
|    raging_wolf_helm|     7717|
|   twitch_recap_2023|     7185|
|   twitch_recap_2024|     7161|
|          subscriber|     7013|
|             partner|     6870|
|      share_the_love|     5656|
|             premium|     5465|
|            no_video|     3685|
|               turbo|     3125|
|            no_audio|     3031|
|        gone_bananas|     2389|
|         glhf_pledge|     2128|
|         rplace_2023|     2088|
|destiny_2_final_s...|     1297|
|superultracombo_2023|     1295|
|       glitchcon2020|     1064|
|      subtember_2024|     1060|
|        bits_charity|      894|
|gold_pixel_heart_...|      869|
|          sub_gifter|      677|
|purple_pixel_hear...|      583|
|                 vip|      518|
|marathon_reveal_r...|      505|
|                bits|      452|
|  chatter

In [39]:
badge_titles.show(100, truncate = False)

+------------------------------------------+---------+
|badge_titles                              |tot_count|
+------------------------------------------+---------+
|Moments Badge - Tier 1                    |42189    |
|Moments Badge - Tier 2                    |19649    |
|Predicted Pink (2)                        |17115    |
|Predicted Blue (1)                        |16794    |
|Moderator                                 |8563     |
|Raging Wolf Helm                          |7717     |
|Twitch Recap 2023                         |7185     |
|Twitch Recap 2024                         |7161     |
|Verified                                  |6870     |
|Share the Love                            |5656     |
|Prime Gaming                              |5465     |
|Subscriber                                |4739     |
|Listening only                            |3685     |
|Moments Badge - Tier 3                    |3627     |
|Turbo                                     |3125     |
|Watching 